In [1]:
import requests
from bs4 import BeautifulSoup
from langchain_cohere import ChatCohere, CohereEmbeddings
from langchain_core.runnables import RunnablePassthrough
from langchain.prompts import PromptTemplate
from langchain_elasticsearch import ElasticsearchStore
from langchain_community.document_transformers import Html2TextTransformer
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.schema import Document  # Import the Document class
from langchain.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser

In [2]:


# Function to load HTML content from a URL
def load_html_content(url):
    response = requests.get(url)
    soup = BeautifulSoup(response.content, 'html.parser')
    return soup.get_text()

# Initialize Cohere embeddings model
embeddings_model = CohereEmbeddings(
    model="embed-english-light-v2.0", cohere_api_key="ERQgasyWTOr2fPq4NQOyHkPxAFM90cgH1oOknZft"
)

# Load documents from URLs
urls = ["https://americanliterature.com/author/o-henry/short-story/a-newspaper-story/"]
docs = [Document(page_content=load_html_content(url), metadata={"source": url}) for url in urls]

# Transform HTML documents to text (if necessary)
html2text = Html2TextTransformer()
docs_transformed = html2text.transform_documents(docs)

# Split documents into smaller chunks
splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=0)
splits = splitter.split_documents(docs_transformed)

# Generate embeddings for the document chunks
split_texts = [doc.page_content for doc in splits]
doc_embeddings = embeddings_model.embed_documents(split_texts)

# Initialize the Elasticsearch store with the embeddings and document chunks
vector_db = ElasticsearchStore.from_documents(
    documents=splits,
    embedding=embeddings_model,
    index_name='webscraping',
    es_cloud_id="webscraping:dXMtZWFzdC0yLmF3cy5lbGFzdGljLWNsb3VkLmNvbTo0NDMkMWJhODg3YmQ5ZDkwNDAzZDgwZTI1ODkyOTA2YzFjYmYkZWZkMTk2MjE3NjQ3NDBkZWI5NTY1NjlkODY0NTMzZWM=",
    es_api_key="bkVWYlRwQUJaX3ZvdDItTVk2dWE6VE8zc2RzdXpTbk9MZzdDeGdxUlRBdw=="
)



INFO:httpx:HTTP Request: POST https://api.cohere.com/v1/embed "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://api.cohere.com/v1/embed "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://api.cohere.com/v1/embed "HTTP/1.1 200 OK"


In [3]:
# used for the retriving of the data base

In [4]:
vector_db = ElasticsearchStore(
    embedding=embeddings_model,
    index_name='webscraping',
    es_cloud_id="webscraping:dXMtZWFzdC0yLmF3cy5lbGFzdGljLWNsb3VkLmNvbTo0NDMkMWJhODg3YmQ5ZDkwNDAzZDgwZTI1ODkyOTA2YzFjYmYkZWZkMTk2MjE3NjQ3NDBkZWI5NTY1NjlkODY0NTMzZWM=",
    es_api_key="bkVWYlRwQUJaX3ZvdDItTVk2dWE6VE8zc2RzdXpTbk9MZzdDeGdxUlRBdw=="
)
# Create a retriever from the Elasticsearch store
retriever = vector_db.as_retriever()

# Define the prompt template
template = '''Answer the question based on the following context.
If the answer does not contain any relevant information to the user query, do not make something up and just say "I don't know!" without any description.
<context>
{context}
</context>

Human Question :: {input}
'''

# Initialize the ChatPromptTemplate with the template
prompt = ChatPromptTemplate.from_template(template)

# Initialize the ChatCohere instance with the Cohere API key
llm = ChatCohere(cohere_api_key="ERQgasyWTOr2fPq4NQOyHkPxAFM90cgH1oOknZft")

# Define the output parser
output_parser = StrOutputParser()

# Define the pipeline
chain = (
    {"context": retriever, "input": RunnablePassthrough()}
    | prompt
    | llm
    | output_parser
)

In [5]:
# Define the query
query = "tell me a story of ant"

# Invoke the pipeline with the query
response = chain.invoke(query)

# Print the response
print(response)

Once upon a time, there was a proud elephant who enjoyed bullying smaller animals. He would go to the anthill near his home and spray water at the ants, knowing they couldn't do anything because of their size. The elephant would laugh and threaten to crush the ants. However, the ants decided to teach the elephant a lesson. They crawled into the elephant's trunk and bit him, causing the elephant to howl in pain. Realizing his mistake, the elephant apologized to the ants and all the animals he had bullied. The moral of the story is to be humble and treat everyone with kindness, regardless of their size or strength.


In [6]:
# Define the query
query = "tell me a story of news paper "

# Invoke the pipeline with the query
response = chain.invoke(query)

# Print the response
print(response)

Here is a short story about a newspaper:

Once upon a time, there was a young boy named Johnny who attended a private school. He often got into trouble with his teacher and frequently faced corporal punishment. One day, Johnny came up with a clever plan to protect himself. He took the newspaper, which featured an editorial against corporal punishment, and carefully cut out columns. He then hid these columns inside his clothes, placing them over areas commonly targeted during beatings. The ink from the newspaper, with its colorful and bold headlines, successfully defended Johnny from any attacks on those areas. 

The power of the press was truly demonstrated that day, as the newspaper's words not only influenced public opinion but also quite literally shielded Johnny from harm. This story showcases how something as simple as a newspaper can have a significant impact and even become a tool for protection and change.


In [7]:
# Define the query
query = "what is the moral of elphant and ant story and there names"

# Invoke the pipeline with the query
response = chain.invoke(query)

# Print the response
print(response) 

The moral of the story is "Be humble and treat everyone with kindness. If you think you’re stronger than others, use your strength to protect them instead of harming them." The elephant and the ants do not have names in this story.


In [8]:

# Define the query
query = "what is anthill in elephant and story ?"

# Invoke the pipeline with the query
response = chain.invoke(query)

# Print the response
print(response)

An anthill is a mound of earth and debris built by ants as their home. In the story "The Elephant and the Ants," the elephant would go to the anthill near his home and spray water at the ants, bullying them due to their smaller size.


In [9]:

# Define the query
query = "what are the all animals are there in all storyes?"

# Invoke the pipeline with the query
response = chain.invoke(query)

# Print the response
print(response)

Sheep and wolves.


In [10]:

# Define the query
query = "tell me all small story on wolf and moral"

# Invoke the pipeline with the query
response = chain.invoke(query)

# Print the response
print(response)

There is a short story about a boy who cried wolf. 

**The Boy Who Cried Wolf**

There was once a shepherd boy who liked to play tricks. One day, while watching over the herd, he cried out “Wolf! Wolf!”, causing people to rush to his aid, only to find that there was no wolf and the boy had played a trick on them. The boy repeated this prank the next day, with the same result. On the third day, the boy actually saw a wolf devouring one of his sheep, and cried for help. However, those who heard him assumed it was another prank, and so no one came to help. The boy lost some of his sheep to the wolf. 

**Moral of the Story:**

If you always lie and cheat on other people, there will come a time when no one will believe you anymore.


In [11]:
## add loggins and monitoring

In [12]:
import logging
import re
import requests
from bs4 import BeautifulSoup
from langchain_cohere import ChatCohere, CohereEmbeddings
from langchain_core.runnables import RunnablePassthrough
from langchain.prompts import PromptTemplate
from langchain_elasticsearch import ElasticsearchStore
from langchain_community.document_transformers import Html2TextTransformer
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.schema import Document  # Import the Document class
from langchain.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer


In [13]:
# Configure logging
logging.basicConfig(level=logging.INFO)
logger = logging.getLogger(__name__)

# Function to load HTML content from a URL
def load_html_content(url):
    try:
        response = requests.get(url)
        response.raise_for_status()  # Raise an HTTPError for bad responses
        soup = BeautifulSoup(response.content, 'html.parser')
        logger.info(f"Loaded content from URL: {url}")
        return soup.get_text()
    except requests.exceptions.RequestException as e:
        logger.error(f"Error loading URL {url}: {e}")
        return ""

# Initialize Cohere embeddings model
embeddings_model = CohereEmbeddings(
    model="embed-english-light-v2.0", cohere_api_key="ERQgasyWTOr2fPq4NQOyHkPxAFM90cgH1oOknZft"
)
logger.info("Initialized Cohere embeddings model")

# Load documents from URLs
urls = ["https://americanliterature.com/author/o-henry/short-story/a-newspaper-story/"]
docs = [Document(page_content=load_html_content(url), metadata={"source": url}) for url in urls]
logger.info("Loaded documents from URLs")

# Transform HTML documents to text (if necessary)
html2text = Html2TextTransformer()
docs_transformed = html2text.transform_documents(docs)
logger.info("Transformed HTML documents to text")

# Split documents into smaller chunks
splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=0)
splits = splitter.split_documents(docs_transformed)
logger.info("Split documents into smaller chunks")

# Generate embeddings for the document chunks
split_texts = [doc.page_content for doc in splits]
doc_embeddings = embeddings_model.embed_documents(split_texts)
logger.info("Generated embeddings for the document chunks")

# Initialize the Elasticsearch store with the embeddings and document chunks
vector_db = ElasticsearchStore.from_documents(
    documents=splits,
    embedding=embeddings_model,
    index_name='webscraping',
    es_cloud_id="webscraping:dXMtZWFzdC0yLmF3cy5lbGFzdGljLWNsb3VkLmNvbTo0NDMkMWJhODg3YmQ5ZDkwNDAzZDgwZTI1ODkyOTA2YzFjYmYkZWZkMTk2MjE3NjQ3NDBkZWI5NTY1NjlkODY0NTMzZWM=",
    es_api_key="bkVWYlRwQUJaX3ZvdDItTVk2dWE6VE8zc2RzdXpTbk9MZzdDeGdxUlRBdw=="
)
logger.info("Initialized Elasticsearch store with the embeddings and document chunks")


INFO:__main__:Initialized Cohere embeddings model
INFO:__main__:Loaded content from URL: https://americanliterature.com/author/o-henry/short-story/a-newspaper-story/
INFO:__main__:Loaded documents from URLs
INFO:__main__:Transformed HTML documents to text
INFO:__main__:Split documents into smaller chunks
INFO:__main__:Generated embeddings for the document chunks
INFO:elastic_transport.transport:GET https://1ba887bd9d90403d80e25892906c1cbf.us-east-2.aws.elastic-cloud.com:443/ [status:200 duration:1.200s]
INFO:elastic_transport.transport:HEAD https://1ba887bd9d90403d80e25892906c1cbf.us-east-2.aws.elastic-cloud.com:443/webscraping [status:200 duration:0.284s]
INFO:elastic_transport.transport:PUT https://1ba887bd9d90403d80e25892906c1cbf.us-east-2.aws.elastic-cloud.com:443/_bulk?refresh=true [status:200 duration:0.625s]
INFO:__main__:Initialized Elasticsearch store with the embeddings and document chunks


In [14]:

# Create a retriever from the Elasticsearch store
retriever = vector_db.as_retriever()

# Advanced Query Handling

stop_words = set(stopwords.words('english'))
ps = PorterStemmer()

def preprocess_query(query):
    words = query.split()
    words = [ps.stem(word) for word in words if word not in stop_words]
    return ' '.join(words)

# Define the prompt template
template = '''Answer the question based on the following context.
If the answer does not contain any relevant information to the user query, do not make something up and just say "I don't know!" without any description.
<context>
{context}
</context>

Human Question :: {input}
'''

# Initialize the ChatPromptTemplate with the template
prompt = ChatPromptTemplate.from_template(template)

# Initialize the ChatCohere instance with the Cohere API key
llm = ChatCohere(cohere_api_key="ERQgasyWTOr2fPq4NQOyHkPxAFM90cgH1oOknZft")

# Define the output parser
output_parser = StrOutputParser()

# Define the pipeline
chain = (
    {"context": retriever, "input": RunnablePassthrough()}
    | prompt
    | llm
    | output_parser
)



In [15]:
# Define the query and preprocess it
query = "Give me the summary of the newspaper story"
preprocessed_query = preprocess_query(query)
logger.info(f"Preprocessed query: {preprocessed_query}")

# Invoke the pipeline with the preprocessed query
response = chain.invoke(preprocessed_query)

# Print the response
print(response)
logger.info(f"Response: {response}")


INFO:__main__:Preprocessed query: give summari newspap stori
INFO:elastic_transport.transport:POST https://1ba887bd9d90403d80e25892906c1cbf.us-east-2.aws.elastic-cloud.com:443/webscraping/_search?_source_includes=metadata,text [status:200 duration:0.301s]
INFO:httpx:HTTP Request: GET https://api.cohere.com/v1/models?default_only=true&endpoint=chat "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://api.cohere.com/v1/chat "HTTP/1.1 200 OK"
INFO:__main__:Response: A young man driving a buggy is startled by a newspaper that blows against his horse's face. The horse bolts and the buggy crashes, leaving the man injured on the street. He is carried into a nearby brownstone mansion, where a woman tends to him, revealing that she is the one he has been pursuing.


A young man driving a buggy is startled by a newspaper that blows against his horse's face. The horse bolts and the buggy crashes, leaving the man injured on the street. He is carried into a nearby brownstone mansion, where a woman tends to him, revealing that she is the one he has been pursuing.
